# Le petit labyrinthe mdr

Bon, je vais pas décrire tout ce qu'il y a de mauvais, juste quelque conseil

Pour commencer, sépare ton gros labyrinthe.py en plusieurs fonctions claires (lis juste t'es pas obligé ca)

In [1]:
init()                # initialise
while (is_running()): # la boucle avec le booleen de fin de jeu
    event()           # gere les evenement
    update()          # affiche sur l'ecran
close()               # cloture (bon la y'a pas besoin generalement)

NameError: name 'init' is not defined

Apres y'a bien evidemment toutes les questions de portées des variables, logiquement on fait un object pour ca.
Bon, on va pas aller dans tout le délire des différentes phases de l'appli (menu puis jeu), qui sont plusieurs classes qui ont le même style d'implémentation (donc qui herite d'une même class "Phase" par exemple).
Voila par exemple pour la phase du labyrinthe (le jeu en lui même):


In [ ]:
class Labyrinthe:
    
    # init le contexte 
    def __init__(self, fenetre, niveau):
        pygame.time.Clock().tick(30)
        
        self.fenetre = fenetre
        self.niveau = niveau
        
    
    def start():
        init()
        while self.is_running:
            event()
            update()
    
    def init():
        self.perso = Perso(img_folder + "pacman.png", niveau)
        self.is_running = True
        
    def event():
        # evenement du joueur
        for event in pygame.event.get():
            if appui sur echap : 
                self.is_running = False
            
            if appui sur droite:
                deplacer(droite)
                
            # ...
        
        # evenement dans le jeu
        if perso arrivé à la fin :
            self.is_running = False
        
    
    def update():
        fenetre.blit(fond, (0, 0))
        niveau.afficher(fenetre)
        # afficher perso etc
        pygame.display.flip()
        
        
    def deplacer(direction):
        # ...

Au final, l'application ressemblerai à un truc de ce style :


In [ ]:
# plein de truc au depart 

while continuer:
    while menu:
        # le menu
        
    niveau = le choix du menu
    
    # Avant y'avais ca :
    while jeu:
        # le jeu

    # maintenant y'a
    labyrinthe = Labyrinthe(fenetre, niveau) # le __init__
    labyrinthe.start() # la boucle du jeu
    
    # ou en plus beau :    Labyrinthe(fenetre, niveau).start()
    

## Pour le problème des intersections :

Deja, la gestion des intersection est pas bonne, si tu changes une case de ton niveau, tu dois refaire toutes les images des intersections

Je propose de tout refaire cette gestion. Je me dis que y'a même pas besoin de l'image "Let's concentrate : left or right". Soit on l'affiche tout le temps, soit ne l'affiche pas du tout.
A chaque déplacement, le perso doit aller direct à l'intersection (ou parcourir petit à petit tout seul mais c'est plus relou vu que faut faire plusieur tour de boucle pour l'affichage, c'est largement faisable mais plus fastidieux)
Du coup la fonction "deplacer" doit déplacer le joueur direct à l'intersection suivante.


Je vais donc refaire le déplacement.
En commencant par refaire la gestion des directions qu'est trop explicite, trop de copier coller.

Ce truc mdr, ca ressemble a rien :

In [ ]:
# Déplacement vers la droite
if direction == 'droite':
    # Pour ne pas dépasser l'écran
    if self.case_x < (nombre_sprite_cote - 1):
        # On vérifie que la case de destination n'est pas un mur
        if self.niveau.structure[self.case_y][self.case_x + 1] != 'm':
            # Déplacement d'une case
            self.case_x += 1
            # Calcul de la position "réelle" en pixel
            self.x = self.case_x * taille_sprite
    # Image dans la bonne direction
    self.direction = self.droite

# Déplacement vers la gauche
if direction == 'gauche':
    if self.case_x > 0:
        if self.niveau.structure[self.case_y][self.case_x - 1] != 'm':
            # time.sleep( 1 )
            self.case_x -= 1

            self.x = self.case_x * taille_sprite
    self.direction = self.gauche

# Déplacement vers le haut
if direction == 'haut':
    if self.case_y > 0:
        if self.niveau.structure[self.case_y - 1][self.case_x] != 'm':
            # time.sleep( 1 )
            self.case_y -= 1

            self.y = self.case_y * taille_sprite
    self.direction = self.haut

# Déplacement vers le bas
if direction == 'bas':
    if self.case_y < (nombre_sprite_cote - 1):
        if self.niveau.structure[self.case_y + 1][self.case_x] != 'm':
            # time.sleep( 1 )
            self.case_y += 1

            self.y = self.case_y * taille_sprite
    self.direction = self.bas
    print(self.case_x, self.case_y)

On vire les valeur x et y du perso, osef, elles sont importantes que dans l'affichage.


In [ ]:
fenetre.blit(pc.direction, (pc.x, pc.y))  # pc.direction = l'image dans la bonne direction

# deviens : 
fenetre.blit(perso.image, position(perso)) # plus besoin du commentaire inutile

def position(perso):
    return (perso.case_x * taille_sprite, perso.case_y * taille_sprite)


Ensuite on gere les direction dans une structure qui permet de mieux les gerer. Normalement j'aurai fait un objet mais bon c'est pas necessaire

In [ ]:
# dans les constantes :

UP, DOWN, LEFT, RIGHT = range(4)   # mon enum des Directions
directions = (UP, DOWN, LEFT, RIGHT) # utile pour les lister
movement = ((0, -1), (0, 1), (-1, 0), (1, 0))

# du coup movement[UP] renvoie (0, -1), c'est a dire un deplacement de 0 en x et de -1 en y


# dans la class Perso :
def move(self, direction):

    self.image = images[direction] # images = [image_haut, image_bas, ...]

    shift_x, shift_y = movement[direction] 

    # les prochaine positions
    next_x = self.case_x + shift_x
    next_y = self.case_y + shift_y

    # si on va en dehors du plateau (on est pas dans le plateau)
    if not (0 <= next_x < nombre_sprite_cote and 0 <= next_y < nombre_sprite_cote):
        return

    # si on se prend un mur
    if self.niveau.structure[next_y][next_x] == 'm':
        return

    # tout est ok, on peut deplacer
    self.case_x = next_x
    self.case_y = next_y

    

On peut donc l'utiliser comme ca :

In [ ]:
# Touches de déplacement 
elif event.key == K_RIGHT:
    perso.move(RIGHT)
elif event.key == K_LEFT:
    perso.move(LEFT)
elif event.key == K_UP:
    perso.move(UP)
elif event.key == K_DOWN:
    perso.move(DOWN)


Voila on a un deplacement propre et lisible, on va faire le parcours d'un chemin vers une intersection.

Mais avant, comme je vois que le deplacement dans un chemin va faire globalemen la meme chose, je vais séparer les differente parties du move() en plusieur fonction:


In [ ]:
def move(self, direction):

    self.image = images[direction] # images = [image_haut, image_bas, ...]

    next_x, next_y = self.next_position(direction)
    
    if self.can_move_to(next_x, next_y):
        self.case_x = next_x
        self.case_y = next_y

def next_position(self, direction):
    shift_x, shift_y = movement[direction] 

    next_x = self.case_x + shift_x
    next_y = self.case_y + shift_y
    return next_x, next_y
    
    
def can_move_to(self, next_x, next_y):
    # si on va en dehors du plateau (on est pas dans le plateau)
    if not (0 <= next_x < nombre_sprite_cote and 0 <= next_y < nombre_sprite_cote):
        return False

    # si on se prend un mur
    if self.niveau.structure[next_y][next_x] == 'm':
        return False
    
    return True

Maintenant on fait le parcours d'un chemin (travel), qu'on utilisera à la place d'un deplacement d'une case (move)

In [ ]:
def travel(self, direction):
    self.move(direction)

    backward_direction = opposite_direction(direction)
    self.continue_to_travel(backward_direction)

def continue_to_travel(self, backward_direction):

    possible_direction = []

    # on test toutes les directions possibles
    for direction in directions:
        next_x, next_y = self.next_position(direction)

        if not self.can_move_to(next_x, next_y):
            continue

        # si c'est la direction d'ou le perso vient
        if direction is backward_direction:
            continue

        # c'est une direction possible
        possible_direction.append(direction)


    # S'il n'y a qu'une seule direction possible
    if len(possible_direction) == 1:
        direction = possible_direction.pop()

        # on continue le parcours
        self.travel(direction)
        


def opposite_direction(direction):
    """ Renvoie la direction opposee (UP <-> DOWN) (LEFT <-> RIGHT) """
    if direction in (UP, LEFT) :
        return directions[direction + 1]
    return directions[direction - 1]


Puis dans le main :

In [ ]:
elif event.key == K_RIGHT:
    perso.travel(RIGHT)
elif event.key == K_LEFT:
    perso.travel(LEFT)
elif event.key == K_UP:
    perso.travel(UP)
elif event.key == K_DOWN:
    perso.travel(DOWN)